# Installations and Setup

In [ ]:
from google.colab import drive

In [ ]:
# TODO: Mount your Google Drive; this allows the runtime environment to access your drive.
drive.mount('/content/gdrive')

# NOTE: Make sure your path does NOT include a '/' at the end!
base_dir = "/content/gdrive/MyDrive/CS4782_Assignments/a4_release"
sys.path.append(base_dir)
## END TODO

In [ ]:
%pip install -r requirements.txt